In [ ]:
import sys
import time
import glob
import os
import datetime
import numpy as np

import pandas as pd
import tensorflow as tf
import tensorflow.keras.losses as kls
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt

from pathlib import Path

gpu_num = 0
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[gpu_num], True)

In [ ]:
main_folder_path = Path('.')
# folder_path = main_folder_path / 'Smaller_data3'
disp = False

class DataGen:
    def __init__(self, batch_size=32, dim_x = 2, dim_y = 1):

        self.bs = batch_size
        self.main_folder_path = Path('.')
        self.folder_path = main_folder_path / 'Smaller_data3'

        self.dim_x = dim_x
        self.dim_y = dim_y
        self.B_max = 0.0
        # self.B_min = 0.0
        self.seq_len = {}

        self.df_seq_len = pd.DataFrame(columns = ['FileName',
                                                  'SeqLength_domain',
                                                  'SeqLength_coil',
                                                  'SeqLength_interface'])

        self.get_sequence_len()
        self.df_seq_len.sort_values(by='SeqLength_domain', ascending=False, inplace=True)

        self.data_size = self.df_seq_len.shape[0]
        self.threshold_points = 300
        # self.norm_denom = self.B_max - self.B_min

        # self.max_seq_len_global = self.df_seq_len.SeqLength.max()


    def get_sequence_len(self):

        files = glob.glob1(self.folder_path, "*.csv")

        for csv_file in files:
            file_path = self.folder_path / csv_file
            df = pd.read_csv(file_path.absolute(), header=None)

            if self.B_max < df[2].max():
                self.B_max = df[2].max()

            counts = df[3].value_counts()

            # 2.0    1904
            # 0.0     248
            # 1.0     113
            # Name: 3, dtype: int64

            record = {'FileName': csv_file,
                      'SeqLength_domain': counts[2],
                      'SeqLength_coil': counts[0],
                      'SeqLength_interface': counts[1]}

            self.df_seq_len = self.df_seq_len.append(record, ignore_index = True)

    def data_feed(self, index):

        if (index + self.bs) >= self.df_seq_len.shape[0]:
            index = self.df_seq_len.shape[0] - self.bs - 1

        # batch_files = list(self.seq_len_sorted)[index : index + self.bs]
        batch_files = self.df_seq_len['FileName'].iloc[index : index + self.bs].tolist()

        max_seq_len_domain = max(self.df_seq_len['SeqLength_domain'].iloc[index : index + self.bs].tolist())
        if max_seq_len_domain > self.threshold_points:
            max_seq_len_domain = self.threshold_points

        max_seq_len_coil = max(self.df_seq_len['SeqLength_coil'].iloc[index : index + self.bs].tolist())
        if max_seq_len_coil > self.threshold_points:
            max_seq_len_coil = self.threshold_points

        max_seq_len_interface = max(self.df_seq_len['SeqLength_interface'].iloc[index : index + self.bs].tolist())
        if max_seq_len_interface > self.threshold_points:
            max_seq_len_interface = self.threshold_points

        max_seq_len_domain += max_seq_len_interface
        max_seq_len_coil += max_seq_len_interface

        src_len = []

        X_domain = np.zeros([self.bs, max_seq_len_domain, self.dim_x])
        X_coil = np.zeros([self.bs, max_seq_len_coil, self.dim_x])
        # X_interface = np.zeros([self.bs, max_seq_len_interface, self.dim_x])

        y_domain = np.zeros([self.bs, max_seq_len_domain])
        y_coil = np.zeros([self.bs, max_seq_len_coil])
        # y_interface = np.zeros([self.bs, max_seq_len_interface])

        mask_domain = np.ones_like(X_domain)
        mask_coil = np.ones_like(X_coil)
        # mask_interface = np.ones_like(X_interface)

        # print(X.shape, y.shape)

        for inx, csv_file in enumerate(batch_files):

            file_path = self.folder_path / csv_file

            df = pd.read_csv(file_path.absolute(), header=None)
            df.rename(columns={0: 'x', 1: 'y', 2: 'B', 3: 'mat'}, inplace=True)

            ### Domain
            seq_len_domain = self.df_seq_len[self.df_seq_len['FileName'] == csv_file]['SeqLength_domain'].iloc[0]
            if seq_len_domain > self.threshold_points:
                seq_len_domain = self.threshold_points

                df_X = df.loc[df['mat'] == 2, ['x', 'y']]
                sample_inx = np.random.randint(0, df_X.shape[0], self.threshold_points)
                X_domain[inx, :seq_len_domain, :] = df_X.iloc[sample_inx, :].values

                df_y = df.loc[df['mat'] == 2, ['B']]
                y_domain[inx, :seq_len_domain] = df_y.iloc[sample_inx, :].values.squeeze()

                mask_domain[inx, seq_len_domain:, :] = 0

            else:
                X_domain[inx, :seq_len_domain, :] = df.loc[df['mat'] == 2, ['x', 'y']].values
                y_domain[inx, :seq_len_domain] = df.loc[df['mat'] == 2, ['B']].values.squeeze()
#                 y_domain /= self.B_max
                mask_domain[inx, seq_len_domain:, :] = 0

            ### Coil
            seq_len_coil = self.df_seq_len[self.df_seq_len['FileName'] == csv_file]['SeqLength_coil'].iloc[0]
            if seq_len_coil > self.threshold_points:
                seq_len_coil = self.threshold_points

                df_X = df.loc[df['mat'] == 0, ['x', 'y']]
                sample_inx = np.random.randint(0, df_X.shape[0], self.threshold_points)
                X_coil[inx, :seq_len_coil, :] = df_X.iloc[sample_inx, :].values

                df_y = df.loc[df['mat'] == 0, ['B']]
                y_coil[inx, :seq_len_coil] = df_y.iloc[sample_inx, :].values.squeeze()
                mask_coil[inx, seq_len_coil:, :] = 0

            else:
                X_coil[inx, :seq_len_coil, :] = df.loc[df['mat'] == 0, ['x', 'y']].values
                y_coil[inx, :seq_len_coil] = df.loc[df['mat'] == 0, ['B']].values.squeeze()
#                 y_coil /= self.B_max
                mask_coil[inx, seq_len_coil:, :] = 0


            ## Interface - new
            seq_len_interface = self.df_seq_len[self.df_seq_len['FileName'] == csv_file]['SeqLength_interface'].iloc[0]
            df_X = df.loc[df['mat'] == 1, ['x', 'y']]
            df_y = df.loc[df['mat'] == 1, ['B']]

            if seq_len_interface > self.threshold_points:
                seq_len_interface = self.threshold_points
                sample_inx = np.random.randint(0, df_X.shape[0], self.threshold_points)
                X_int = df_X.iloc[sample_inx, :].values
                y_int = df_y.iloc[sample_inx, :].values.squeeze()
            else:
                X_int = df.loc[df['mat'] == 1, ['x', 'y']].values
                y_int = df.loc[df['mat'] == 1, ['B']].values.squeeze()

            # Int - domain
            inx_start = seq_len_domain
            inx_stop = seq_len_domain + seq_len_interface

            X_domain[inx, inx_start:inx_stop, :] = X_int
            y_domain[inx, inx_start:inx_stop] = y_int.squeeze()
            y_domain[inx, :] /= self.B_max
            mask_domain[inx, inx_start:inx_stop, :] = 1

            # Int - coil
            inx_start = seq_len_coil
            inx_stop = seq_len_coil + seq_len_interface

            X_coil[inx, inx_start:inx_stop, :] = X_int
            y_coil[inx, inx_start:inx_stop] =  y_int
            y_coil[inx, :] /= self.B_max
            mask_coil[inx, inx_start:inx_stop, :] = 1

            ## Interface - old
            # seq_len_interface = self.df_seq_len[self.df_seq_len['FileName'] == csv_file]['SeqLength_interface'].iloc[0]
            # if seq_len_interface > self.threshold_points:
            #   seq_len_interface = self.threshold_points

            #   df_X = df.loc[df['mat'] == 1, ['x', 'y']]
            #   sample_inx = np.random.randint(0, df_X.shape[0], self.threshold_points)
            #   X_interface[inx, :seq_len_interface, :] = df_X.iloc[sample_inx, :].values

            #   df_y = df.loc[df['mat'] == 1, ['B']]
            #   y_interface[inx, :seq_len_interface] = df_y.iloc[sample_inx, :].values.squeeze()

            #   mask_interface[inx, seq_len_interface:, :] = 0

            # else:
            #   X_interface[inx, :seq_len_interface, :] = df.loc[df['mat'] == 1, ['x', 'y']].values
            #   y_interface[inx, :seq_len_interface] = df.loc[df['mat'] == 1, ['B']].values.squeeze()
            #   # y_interface /= self.B_max
            #   mask_interface[inx, seq_len_interface:, :] = 0

            # src_len.append((seq_len_domain, seq_len_coil, seq_len_interface))

            # masks = [mask_domain, mask_coil, mask_interface]
            # X = [X_domain, X_coil, X_interface]
            # y = [y_domain, y_coil, y_interface]

            src_len.append((seq_len_domain + seq_len_interface, seq_len_coil + seq_len_interface))

            masks = [mask_domain, mask_coil]
            X = [X_domain, X_coil]
            y = [y_domain, y_coil]

        return X, y, masks, src_len

    def save_plots(self, x, y, uex, upred, epoch, comment):

        fig, (ax1, ax2) = plt.subplots(1,2,figsize=(26,10))

        ## True Exact
        # ax2.tricontour(x, y, z, levels=20, linewidths=0.5, colors='k')
        cntr1 = ax1.tricontourf(x, y, uex, levels=20, cmap="jet")

        fig.colorbar(cntr1, ax=ax1)
        # ax2.plot(x, y, 'ko', ms=3)
        # ax1.set(xlim=(-1, 1), ylim=(-1, 1))
        ax1.set_title('Exact')

        ## Predictions
        # ax2.tricontour(x, y, z, levels=20, linewidths=0.5, colors='k')
        cntr2 = ax2.tricontourf(x, y, upred, levels = 20, cmap = 'jet')

        fig.colorbar(cntr2, ax=ax2)
        # ax2.plot(x, y, 'ko', ms=3)
        # ax2.set(xlim=(-1, 1), ylim=(-1, 1))
        ax2.set_title('Prediction')

        plt.subplots_adjust(hspace=0.5)

        file_name = f'epoch_{epoch+1}.png'
        result_folder = self.main_folder_path / 'Results_Sep_RNN' / comment

        if not os.path.exists(result_folder):
            os.makedirs(result_folder)

        plt.savefig(result_folder.absolute() / file_name, dpi=100)

        #plt.savefig(file_name, dpi=100)

        #plt.show()
        plt.close(fig)
        plt.close('all')

    def save_stats(self, dict_, epoch, comment):
       # with open('history %s' % (epoch), 'w') as outfile:
        #    outfile.write(dict_)
        result_folder = self.main_folder_path / 'Results_Sep_RNN' / comment

        if not os.path.exists(result_folder):
            os.makedirs(result_folder)
        file_name = f'epoch_{epoch+1}.csv'

        print(dict_)
        df = pd.DataFrame(dict_, index=[0])
        df.to_csv(result_folder.absolute() / file_name)

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, bi=False):
        super(MyModel, self).__init__()
        self.bi = bi

        if self.bi == True:
            self.lstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                            128, activation='sigmoid',
                           dropout= 0.2, recurrent_dropout=0.2,
                           kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                            return_sequences= True, return_state= True))

            self.BN_1 = tf.keras.layers.BatchNormalization()

            self.lstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                            64, activation='sigmoid',
                           dropout= 0.2, recurrent_dropout=0.2,
                           kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                            return_sequences= True))

            self.BN_2 = tf.keras.layers.BatchNormalization()

        else:
            self.lstm_1 = tf.keras.layers.GRU(
                            128, activation='sigmoid',
                            dropout= 0.2, recurrent_dropout=0.2,
                            kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                            return_sequences= True, return_state= True)

            self.BN_1 = tf.keras.layers.BatchNormalization()

            self.lstm_2 = tf.keras.layers.GRU(
                            64, activation='sigmoid',
                            dropout= 0.2, recurrent_dropout=0.2,
                            kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                            return_sequences= True)

            self.BN_2 = tf.keras.layers.BatchNormalization()

        self.dense = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))

    def call(self, inputs):

        input1 = inputs[0]
        mask = inputs[1]

#        print(mask.shape)
        if self.bi == True:
            x, state_h, state_c = self.lstm_1(input1, mask=mask)
        else:
            x, state_h = self.lstm_1(input1, mask=mask)

        x = self.BN_1(x)
        x = self.lstm_2(x, mask=mask)
        x = self.BN_2(x)
        output = self.dense(x, mask=mask)
        return output

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def my_training(batch, epochs, no_iters, lr, bi, train_summary_writer, comment):

    mean_loss = tf.keras.metrics.Mean()
    metrics = [tf.keras.metrics.mean_absolute_error]
    model = MyModel(bi=bi)
    history = {}
    optimizer = tf.keras.optimizers.Nadam(learning_rate= lr)

    for epoch in range(epochs):
        start_time = time.time()

        for i in range(no_iters):

            index = np.random.randint(batch.data_size-batch.bs)
            X_d, X_c, Y_d, Y_c, mask_d, mask_c, seq_len_d, seq_len_c = batch.data_feed(index)

            # X, Y, mask, src_len = batch.data_feed(index)
#             Y_c *= 1E6
            Y_c = np.expand_dims(Y_c, axis=-1)

#             Y_d *= 1E6
            Y_d = np.expand_dims(Y_d, axis=-1)

            # mask = np.logical_not(X) # Finds all the zero values
            # returns True (before the NOT) if all the elemenst in the row are zero
            # mask = np.logical_not(np.all(mask, axis=-1))

            mask_d = np.logical_not(mask_d) # Finds all the zero values
            # returns True (before the NOT) if all the elemenst in the row are zero
            mask_d = np.logical_not(np.all(mask_d, axis=-1))

            mask_c = np.logical_not(mask_c) # Finds all the zero values
            # returns True (before the NOT) if all the elemenst in the row are zero
            mask_c = np.logical_not(np.all(mask_c, axis=-1))

            X_train = [X_d, mask_d]
            # print(len(X_train))
            # print('X.shape', X_d.shape)
            # print('mask.shape', mask_d.shape)
            # print('X[:,:, 0].shape', X_d[:,:, 0].shape)
            # print('X[:,:, 1].shape',X_d[:,:, 1].shape)
            # print('Y.shape', Y_d.shape)
            # print('Y_d.max', Y_d.max())
            # print('Y_d.min', Y_d.min())

            with tf.GradientTape() as tape:
                y_pred = model(X_train, training=True)
                # print('y_pred.shape', y_pred.shape)
                loss_ = tf.keras.losses.mean_squared_error(Y_d, y_pred)

            gradients = tape.gradient(loss_, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            mean_loss(loss_)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        print('Epoch: {:02} | Time: {}m {}s\t'.format(epoch+1, epoch_mins, epoch_secs))
        try:
            print('step %s: mean loss = %s' % (epoch+1, mean_loss.result().numpy()))
            history[epoch] = mean_loss.result().numpy()
        except:
            print('step %s: mean loss = %s' % (epoch+1, mean_loss.result()))
            history[epoch] = mean_loss.result()

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', mean_loss.result(), step=epoch)

        y_pred = model(X_train, training=False)
        y_pred = y_pred.numpy()
        #print('y_pred.shape', y_pred.shape)
        batch.save_plots(X_d[1, :, 0], X_d[1, :, 1], Y_d[1, :,0], y_pred[1, :, 0], epoch, comment)

        mean_loss.reset_states()
        tf.summary.flush(train_summary_writer)

    # Serializing json
    #history_object = json.dumps(history, indent = 4)
    batch.save_stats(history, epoch, comment)
    #print(history_object)

    return history

In [ ]:
batch_size = 32
epochs = 10
lr = 0.001
bi = False

main_folder_path = Path('.')
folder_path = main_folder_path / 'Smaller_data3'

batch = DataGen(batch_size= batch_size)
batch.folder_path = folder_path

no_iters = int(batch.df_seq_len.shape[0]/batch_size)*2

# index = np.random.randint(batch.df_seq_len.shape[0]-batch.bs)
# # X, Y, _, _ = batch.data_feed(index)
# X_d, X_c, Y_d, Y_c, mask_d, mask_c, seq_len_d, seq_len_c = batch.data_feed(index)

comment = f' App_batch_size={batch_size} lr={lr} Bi-dir={bi}'
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'runs/MagApp/' + current_time + '/train' + comment
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

In [ ]:
# history = my_training(batch, epochs, no_iters, lr, bi, train_summary_writer, comment)
# my_training(batch, epochs, no_iters, lr, bi)

# Separate Models for different Domains

In [ ]:
def learn(batch, epochs, no_iters, lr, bi, train_summary_writer, comment):

    mean_loss_11 = tf.keras.metrics.Mean()
    mean_loss_22 = tf.keras.metrics.Mean()
    mean_loss_12 = tf.keras.metrics.Mean()
    mean_loss_21 = tf.keras.metrics.Mean()

    metrics = [tf.keras.metrics.mean_absolute_error]
    model_domain = MyModel(bi=bi)
    model_coil = MyModel(bi=bi)

    optimizer_domain = tf.keras.optimizers.Nadam(clipvalue = 1, learning_rate= lr)
    optimizer_coil = tf.keras.optimizers.Nadam(clipvalue = 1, learning_rate= lr)

    for epoch in range(epochs):
        start_time = time.time()

        for i in range(no_iters):

            index = np.random.randint(batch.data_size-batch.bs)

            # X_domain, X_coil, y_domain, y_coil, mask_domain, mask_coil, seq_len_domain + seq_len_interface, seq_len_coil + seq_len_interface


            X, Y, masks, src_len = batch.data_feed(index)
            Y = [np.expand_dims(y*1E0, axis=-1) for y in Y]
            # Y_domain, Y_coil, Y_interface = Y
            Y_domain, Y_coil = Y

            masks = [np.logical_not(mask) for mask in masks]
            masks = [np.logical_not(np.all(mask, axis=-1)) for mask in masks]

            # X_domain, X_coil, X_interface = [[x, mask] for x, mask in zip(X, masks)]
            X_domain, X_coil = [[x, mask] for x, mask in zip(X, masks)]

            with tf.GradientTape() as tape1 :
                y_pred_domain = model_domain(X_domain, training=True)
                # y_pred_1_int = model_domain(X_interface, training=True)

                loss_domain = tf.keras.losses.mean_squared_error(Y_domain, y_pred_domain)
                # loss_1_int = tf.keras.losses.mean_squared_error(Y_interface, y_pred_1_int)

            # grads11 = tape1.gradient(loss_domain, model_domain.trainable_variables)
            # grads12 = tape1.gradient(loss_1_int, model_domain.trainable_variables)

            # optimizer_domain.apply_gradients(zip(grads11, model_domain.trainable_variables))
            # optimizer_domain.apply_gradients(zip(grads12, model_domain.trainable_variables))

            # grads1 = tape1.gradient([loss_domain, loss_1_int], model_domain.trainable_variables)
            grads1 = tape1.gradient(loss_domain, model_domain.trainable_variables)
            optimizer_domain.apply_gradients(zip(grads1, model_domain.trainable_variables))

            with tf.GradientTape() as tape2 :
                y_pred_coil = model_coil(X_coil, training=True)
                # y_pred_2_int = model_coil(X_interface, training=True)

                loss_coil = tf.keras.losses.mean_squared_error(Y_coil, y_pred_coil)
                # loss_2_int = tf.keras.losses.mean_squared_error(Y_interface, y_pred_2_int)

            grads22 = tape2.gradient(loss_coil, model_coil.trainable_variables)
            # grads21 = tape2.gradient(loss_2_int, model_coil.trainable_variables)

            optimizer_coil.apply_gradients(zip(grads22, model_coil.trainable_variables))
            # optimizer_coil.apply_gradients(zip(grads21, model_coil.trainable_variables))

            mean_loss_11(loss_domain)
            mean_loss_22(loss_coil)
            # mean_loss_12(loss_1_int)
            # mean_loss_21(loss_2_int)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        print('Epoch: {:02} | Time: {}m {}s\t'.format(epoch+1, epoch_mins, epoch_secs))
        try:
            # print(f'step {epoch+1}, \
            # mean loss 11 = {mean_loss_11.result().numpy():,.2f}, \
            # mean loss 12 = {mean_loss_12.result().numpy():,.2f}, \
            # mean loss 22 = {mean_loss_22.result().numpy():,.2f},\
            # mean loss 21 = {mean_loss_21.result().numpy():,.2f}')

            print(f'step {epoch+1}, \
            mean loss 11 = {mean_loss_11.result().numpy():,.2f}, \
            mean loss 22 = {mean_loss_22.result().numpy():,.2f}')
        except:
            # print(f'step {epoch+1}, \
            # mean loss 11 = {mean_loss_11.result():,.2f} \
            # mean loss 11 = {mean_loss_12.result():,.2f} \
            # mean loss 11 = {mean_loss_22.result():,.2f} \
            # mean loss 11 = {mean_loss_21.result():,.2f}')

            print(f'step {epoch+1}, \
            mean loss 11 = {mean_loss_11.result():,.2f} \
            mean loss 11 = {mean_loss_22.result():,.2f}')

        # with train_summary_writer.as_default():
        #     tf.summary.scalar('loss', mean_loss.result(), step=epoch)

    #         try:
    #             for lay in model.layers:
    #                 lay_weights = lay.get_weights()
    #                 lay_name = lay.name
    #                 for idx, lay_wnb in enumerate(lay_weights):
    # #                    print(f'{lay_name}_{idx}', lay_wnb.shape)
    #                     tf.summary.histogram(f'{lay_name}_{idx}', lay_wnb, epoch)
    #         except:
    #             pass

        # y_pred = model(X_train, training=False)
        # y_pred = y_pred.numpy()

        # mean_loss.reset_states()
        # tf.summary.flush(train_summary_writer)

        with train_summary_writer.as_default():
            tf.summary.scalar('loss_domain', mean_loss_11.result(), step=epoch)
            tf.summary.scalar('loss_coil', mean_loss_22.result(), step=epoch)

        X_d = X_domain[0]
        X_c = X_coil[0]
        Y_d, Y_c = Y

        X_print = np.concatenate((X_d[0], X_c[0]), axis=0)
        y_print = np.concatenate((Y_d[0], Y_c[0]), axis=0)

#         print("y_print============================")
#         print(y_print)

        y_pred_domain = model_domain(X_domain, training=False)
        y_pred_domain = y_pred_domain.numpy()

        y_pred_coil = model_coil(X_coil, training=False)
        y_pred_coil = y_pred_coil.numpy()

#         print("y_pred_domain====================")
#         print(y_pred_domain)

        y_pred = np.concatenate((y_pred_domain[0], y_pred_coil[0]), axis=0)

#         print("y_pred===========================")
#         print(y_pred)

        #print('y_pred.shape', y_pred.shape)
        batch.save_plots(X_print[:, 0], X_print[:, 1], y_print[:,0], y_pred[:, 0], epoch, comment)

        mean_loss_11.reset_states()
        mean_loss_22.reset_states()
        tf.summary.flush(train_summary_writer)

    # Serializing json
    #history_object = json.dumps(history, indent = 4)
    batch.save_stats(history, epoch, comment)
    #print(history_object)

    return history

In [ ]:
# learn(batch, epochs, no_iters, lr, bi)
history = learn(batch, epochs, no_iters, lr, bi, train_summary_writer, comment)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 01 | Time